Номер последнего патента на срок 12.03.2023: 2789828

Номер патента, с которого я могу парсить: 2005333

Количество патентов, которые можно спарсить: 784495

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def parse_patent(html_code):
    """Функция собирает всю необходимую информацию о патенте с html-кода.

    Args:
        html_code (lxml): html-код страницы патента 

    Returns:
        list: [
                number - Номер патента,
                date - дата подачи заявки,
                quotes - список цитированных в отчёте документов,
                authors - авторы,
                patent_owner - патентообладатель,
                mpk - МПК,
                spk - СПК,
                country - страна,
                type_of_document - тип документа,
                title - название патента,
                abstract - реферат,
                patent_claims - формула патента,
                patent_description - описание патента,
                sources_of_information - источники информации,
                ]

    """

    if html_code.text == 'Документ с данным номером отсутствует':
        return 'Does not exist'

    keys_data_1 = html_code.find('table', id='bib').find('tr').findAll('td')[0].findAll('p')
    keys_data_2 = html_code.find('table', id='bib').find('tr').findAll('td')[1].findAll('p')

    list_date = [i for i in keys_data_1 if '(21)(22)' in i.text]
    if list_date:
        date = list_date[0].find('b').text.split(', ')[-1] # Дата подачи заявки
    else:
        date = []

    list_quotes = [i for i in keys_data_1 if '(56)' in i.text]
    if list_quotes:
        quotes = list_quotes[0].find('b').text.split('. ') # Список документов, цитированных в отчете о поиске
    else:
        quotes = []

    list_authors = [i for i in keys_data_2 if '(72)' in i.text]
    if list_authors:
        authors = list_authors[0].find('b').text[1:].split(',') # Автор(ы)
    else:
        authors = []

    list_patent_owner = [i for i in keys_data_2 if '(73)' in i.text]
    if list_patent_owner:
        patent_owner = keys_data_2[1].find('b').text[1:] # Патентообладатель(и)
    else:
        patent_owner = []

    keys_data_3 = html_code.find('table', class_='tp').findAll('tr')
    mpk = [' '.join(i.text[1:-1].split()) for i in keys_data_3[3].find('div').find('ul').findAll('li')] # МПК
    try:
        spk = [' '.join(i.text[1:-1].split()) for i in keys_data_3[5].find('div').find('ul').findAll('li')] # СПК
    except AttributeError:
        spk = [keys_data_3[5].text]

    keys_data_4 = html_code.find('table', class_='tp').findAll('div', class_='topfield2')
    country = keys_data_4[0].text # Страна
    number = keys_data_4[1].text[1:-1].replace(" ", "") # Номер патента
    type_of_document = keys_data_4[2].text # Тип документа

    title = ' '.join(html_code.find('p', id='B542').text.split()[1:]) # Название патента
    abstract = html_code.find('div', id='Abs').findAll('p')[1].text[:-1] # Реферат патента

    ind_abstract = [ind for ind, i in enumerate(html_code.findAll('p')) if '(57)' in i.text][0] # Индекс реферата

    trash = set(['', ' ', '  ', '\n', '\n,', '\n, ', '\n\n', '\n\n,', '\n\n, ', '\n\n\n', '\n\n\n,', '\n\n\n ', '\n\n\n, ', '\n\n\n\n']) # Возможный мусор
    all_text = [i.text for i in html_code.findAll('p')[ind_abstract+2:] if not (i.text in trash)] # Весь патент

    ind_source_of_inf = [ind for ind, i in enumerate(all_text) if 'Источники информации' in i] # Индекс "Источники информации"
    ind_patent_claims = [ind for ind, i in enumerate(all_text) if 'Формула изобретения' in i][0] # Индекс "Формула патента"

    if ind_source_of_inf:
        patent_description = all_text[:ind_source_of_inf[0]] # Описание патента
        source_of_information = all_text[ind_source_of_inf[0]+1:ind_patent_claims] # Источники информации
    else:
        patent_description = all_text[:ind_patent_claims]
        source_of_information = [] # Источники информации

    ind_notification = [ind for ind, i in enumerate(all_text) if 'ИЗВЕЩЕНИЯ' in i] # Индекс "ИЗВЕЩЕНИЯ
    if ind_notification:
        patent_claims = all_text[ind_patent_claims+1:ind_notification[0]] # Формула патента
    else:
        patent_claims = all_text[ind_patent_claims+1:] # Формула патента

    return [number, date, quotes, authors, patent_owner, mpk, spk, country, type_of_document, title, abstract, patent_claims, patent_description, source_of_information]

In [62]:
headers = ['number','date', 'quotes', 'authors', 'patent_owner', 'mpk', 'spk', 'country', 'type_of_document', 'title', 'abstract', 'patent_claims', 'patent_description', 'source_of_information']
infos = pd.DataFrame(infos_about_patents, columns=headers)
infos.to_csv('about_patents1.csv')

In [9]:
import time
start_time = time.time()

time.sleep(3)
end_time = time.time()
round(end_time-start_time, 2)

3.01

In [6]:
def parse_patent(html_code, number_patent):
    """Функция собирает всю необходимую информацию о патенте с html-кода.

    Args:
        html_code (lxml): html-код страницы патента
        number_patent (str): номер патента, который сейчас парсится

    Returns:
        list: [
                number - Номер патента,
                date - дата подачи заявки,
                quotes - список цитированных в отчёте документов,
                authors - авторы,
                patent_owner - патентообладатель,
                mpk - МПК,
                spk - СПК,
                country - страна,
                type_of_document - тип документа,
                title - название патента,
                abstract - реферат,
                patent_claims - формула патента,
                patent_description - описание патента,
                sources_of_information - источники информации,
                ]

    """

    if html_code.text == "Документ с данным номером отсутствует":
        data = [""] * 14
        data[0] = number_patent
        return (data, True)

    try:
        keys_data_1 = (
            html_code.find("table", id="bib").find("tr").findAll("td")[0].findAll("p")
        )
        keys_data_2 = (
            html_code.find("table", id="bib").find("tr").findAll("td")[1].findAll("p")
        )

        list_date = [i for i in keys_data_1 if "(21)(22)" in i.text]
        if list_date:
            date = list_date[0].find("b").text.split(", ")[-1]  # Дата подачи заявки
        else:
            date = []

        list_quotes = [i for i in keys_data_1 if "(56)" in i.text]
        if list_quotes:
            quotes = (
                list_quotes[0].find("b").text.split(". ")
            )  # Список документов, цитированных в отчете о поиске
        else:
            quotes = []

        list_authors = [i for i in keys_data_2 if "(72)" in i.text]
        if list_authors:
            authors = list_authors[0].find("b").text[1:].split(",")  # Автор(ы)
        else:
            authors = []

        list_patent_owner = [i for i in keys_data_2 if "(73)" in i.text]
        if list_patent_owner:
            patent_owner = keys_data_2[1].find("b").text[1:]  # Патентообладатель(и)
        else:
            patent_owner = []

        keys_data_3 = html_code.find("table", class_="tp").findAll("tr")
        mpk = [
            " ".join(i.text[1:-1].split())
            for i in keys_data_3[3].find("div").find("ul").findAll("li")
        ]  # МПК
        try:
            spk = [
                " ".join(i.text[1:-1].split())
                for i in keys_data_3[5].find("div").find("ul").findAll("li")
            ]  # СПК
        except AttributeError:
            spk = [keys_data_3[5].text]

        keys_data_4 = html_code.find("table", class_="tp").findAll(
            "div", class_="topfield2"
        )
        country = keys_data_4[0].text  # Страна
        number = keys_data_4[1].text[1:-1].replace(" ", "")  # Номер патента
        type_of_document = keys_data_4[2].text  # Тип документа

        title = " ".join(
            html_code.find("p", id="B542").text.split()[1:]
        )  # Название патента
        abstract = (
            html_code.find("div", id="Abs").findAll("p")[1].text[:-1]
        )  # Реферат патента

        ind_abstract = [
            ind for ind, i in enumerate(html_code.findAll("p")) if "(57)" in i.text
        ][
            0
        ]  # Индекс реферата

        trash = set(
            [
                "",
                " ",
                "  ",
                "\n",
                "\n,",
                "\n, ",
                "\n\n",
                "\n\n,",
                "\n\n, ",
                "\n\n\n",
                "\n\n\n,",
                "\n\n\n ",
                "\n\n\n, ",
                "\n\n\n\n",
            ]
        )  # Возможный мусор
        all_text = [
            i.text
            for i in html_code.findAll("p")[ind_abstract + 2 :]
            if not (i.text in trash)
        ]  # Весь патент

        ind_source_of_inf = [
            ind for ind, i in enumerate(all_text) if "Источники информации" in i
        ]  # Индекс "Источники информации"
        ind_patent_claims = [
            ind for ind, i in enumerate(all_text) if "Формула изобретения" in i
        ][
            0
        ]  # Индекс "Формула патента"

        if ind_source_of_inf:
            patent_description = all_text[: ind_source_of_inf[0]]  # Описание патента
            source_of_information = all_text[
                ind_source_of_inf[0] + 1 : ind_patent_claims
            ]  # Источники информации
        else:
            patent_description = all_text[:ind_patent_claims]
            source_of_information = []  # Источники информации

        ind_notification = [
            ind for ind, i in enumerate(all_text) if "ИЗВЕЩЕНИЯ" in i
        ]  # Индекс "ИЗВЕЩЕНИЯ
        if ind_notification:
            patent_claims = all_text[
                ind_patent_claims + 1 : ind_notification[0]
            ]  # Формула патента
        else:
            patent_claims = all_text[ind_patent_claims + 1 :]  # Формула патента

        return ([
            number,
            date,
            quotes,
            authors,
            patent_owner,
            mpk,
            spk,
            country,
            type_of_document,
            title,
            abstract,
            ' '.join(patent_claims),
            ' '.join(patent_description),
            source_of_information,
        ], True)
    except:
        data = [""] * 14
        data[0] = number_patent
        return (data, False)

In [4]:
from utils import parse_patent

In [5]:
start_number = 2000000
link = f"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber={start_number}&TypeFile=html"

r = requests.get(link)
soup = BeautifulSoup(r.text, 'lxml')

In [6]:
parse_patent(soup, r.url.split('&')[1].split('=')[1])

(['2000000', '', '', '', '', '', '', '', '', '', '', '', '', ''], True)

In [14]:
numb = 2640290
link = f"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber={numb}&TypeFile=html"

r = requests.get(link)
soup = BeautifulSoup(r.text, 'lxml')

In [35]:
parse_patent(soup, r.url.split('&')[1].split('=')[1])

['2640290',
 '29.03.2017',
 ['RU 2612325 C1, 07.03.2017',
  'RU 2247456 C2, 27.02.2005',
  'RU 2096885 C1, 20.11.1997',
  'US 5796258 A, 18.08.1998.'],
 ['Лямец Юрий Яковлевич (RU)',
  'Романов Юрий Вячеславович (RU)',
  'Нестерин Андрей Алексеевич (RU)'],
 'Общество с ограниченной ответственностью "Релематика" (RU)',
 ['H02H 7/06 (2006.01)'],
 ['H02H 7/06 (2006.01)', 'H02H 3/40 (2006.01)'],
 'RU',
 'C1',
 'СПОСОБ РЕЛЕЙНОЙ ЗАЩИТЫ ГЕНЕРАТОРА',
 'Использование: в области электротехники и электроэнергетики. Технический результат заключается в расширении функциональных возможностей и в упрощении способа. Генератор наблюдают со стороны линейных и нулевых выводов. Фиксируют момент смены предшествующего режима текущим режимом. Алгоритмическую модель активируют источниками напряжений текущего режима. Определяют ее реакцию в виде первых токов обмотки статора. Если генератор не поврежден, то первые токи будут близки к наблюдаемым, так как модель в этом случае адекватна реальному объекту. В случа

In [12]:
start_number = 2640290
link = f"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber={start_number}&TypeFile=html"

infos_about_patents = []
i = 0
while True:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    parsed_info = parse_patent(soup, r.url.split('&')[1].split('=')[1])
    infos_about_patents.append(parsed_info)
    start_number += 1
    link = f"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber={start_number}&TypeFile=html"

    i += 1
    if i == 10:
        break

    time.sleep(2)

In [13]:
[i[1] for i in infos_about_patents]

[True, True, True, True, True, True, False, True, True, True]

In [39]:
import csv

In [40]:
with open("data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    header = [
        "number",
        "date",
        "quotes",
        "authors",
        "patent_owner",
        "mpk",
        "spk",
        "country",
        "type_of_document",
        "title",
        "abstract",
        "patent_claims",
        "patent_description",
        "source_of_information",
    ]
    writer.writerows([header])

In [26]:
def fetch_url(link):
    r = requests.get(link)
    number_of_patent = link.split('=')[2].split('&')[0]
    soup = BeautifulSoup(r.text, 'lxml')
    parsed_info = parse_patent(soup, number_of_patent)
    return parsed_info

In [45]:
start_number = 2006534
n_batch = 60
links = [
    f"https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber={i}&TypeFile=html"
    for i in range(start_number, start_number + n_batch)
]
links

['https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006534&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006535&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006536&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006537&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006538&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006539&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006540&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006541&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006542&TypeFile=html',
 'https://new.fips.ru/registers-doc-view/fips_servlet?DB=RUPAT&DocNumber=2006543&TypeFile=html',
 'https://new.fips.ru/register

In [46]:
infos_ab_patents = []
while links:
    list_patents = []
    for link in links:
        patent_info = fetch_url(link)
        list_patents.append(patent_info)
        time.sleep(2)
    mask = [i[1] for i in list_patents]
    
    list_ab_patents = []
    list_links = []
    for i in range(len(mask)):
        if mask[i]:
            list_ab_patents.append(list_patents[i][0])
        else:
            list_links.append(links[i])
    
    infos_ab_patents.extend(list_ab_patents)
    links = list_links.copy()
    
    print(mask)

[True, True, True, True, True, True, False, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, False, True, True, True, False, True, True, True, True, False, True, True, True, False, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, True, True, False, True, True, True, False]
[True, True, False, True, True, True, True, True, True, True, True]
[False]
[True]


In [23]:
[i[1] for i in infos_about_patents]

[True, True, True, True, True, True, False, True, True, True]

In [34]:
l1 = []
l2 = [2, 2]
l1.extend(l2)
print(l1)

[2, 2]


In [49]:
27/60*784495/60/60

98.061875

In [51]:
from random_user_agent.user_agent import UserAgent

In [8]:
user_agent_rotator = UserAgent(limits = 5)

NameError: name 'UserAgent' is not defined

In [7]:
[{'User-Agent':user_agent_rotator.get_random_user_agent()} for i in range(5)]

NameError: name 'user_agent_rotator' is not defined

In [54]:
user_agent_rotator.get_random_user_agent()

'Mozilla/5.0 (Linux; Android 4.1.1; Positivo Ypy AB7EC Build/JRO03H) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166  Safari/535.19'

In [55]:
user_agent_rotator.get_random_user_agent()

'UCWEB/2.0 (MIDP-2.0; U; Adr 5.1; en-US; Micromax_Q346) U2/1.0.0 UCBrowser/10.1.5.583 U2/1.0.0 Mobile'

In [9]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem


# you can also import SoftwareEngine, HardwareType, SoftwareType, Popularity from random_user_agent.params
# you can also set number of user agents required by providing `limit` as parameter

software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

# Get list of user agents.
user_agents = user_agent_rotator.get_user_agents()

# Get Random User Agent String.
user_agent = user_agent_rotator.get_random_user_agent()

In [15]:
user_agent_rotator.get_random_user_agent()

'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2018.0 Safari/537.36'